# Estymacja cen opcji europejskich i barierowych metodą Monte Carlo
## Wprowadzenie do symulacji i metod Monte Carlo
### Mikołaj Płóciniczak, 2026

In [3]:
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter
from IPython.display import HTML

def show_code(path):
    code = open(path).read()
    formatter = HtmlFormatter(style="native")  # działa dobrze w trybie ciemnym
    styles = formatter.get_style_defs('.highlight')
    html = f"<style>{styles}</style>" + highlight(code, PythonLexer(), formatter)
    return HTML(html)


# 1. Wprowadzenie

Celem tego raportu jest estymacja cen opcji europejskiej oraz dyskretnej opcji barierowej typu up-and-out metodą Monte Carlo. Rozważamy proces ceny akcji opisany geometrycznym ruchem Browna (GBM) o postaci
$$
S(t) = S(0)\exp(\mu^* t + \sigma B(t)),
$$
gdzie $B(t)$ jest ruchem Browna, a parametry modelu są ustalone: $r = 0{,}05$, $\sigma = 0{,}125$, $\mu^* = r - \sigma^2/2 = -0{,}0125$, $S(0) = 100$ oraz $K = 100$.

W projekcie porównujemy kilka estymatorów Monte Carlo:
- klasyczny estymator typu Crude Monte Carlo,
- estymator ze stratyfikacją (dla podziału proporcjonalnego i optymalnego),
- estymator z antytetycznymi zmiennymi (dla $n = 1$),
- estymator z wykorzystaniem zmiennej kontrolnej (dla $n = 1$).

Dla przypadku opcji europejskiej ($C = \infty$) wyniki symulacji porównujemy z wartością dokładną daną wzorem Blacka–Scholesa. Dla skończonych wartości bariery $C$ analizujemy wpływ bariery na wartość opcji oraz efektywność poszczególnych technik redukcji wariancji.

# Metody symulacyjne

W tej części opisujemy sposób generowania ścieżek procesu GBM oraz konstrukcję różnych estymatorów Monte Carlo użytych w projekcie.

Poniżej przedstawiono krótkie omówienie modeli oraz metod, na których opiera się dalsza analiza.

## 2.1. Model geometrycznego ruchu Browna

Model geometrycznego ruchu Browna (GBM) opisuje ewolucję ceny aktywa jako
$$
S(t) = S(0)\exp(\mu^* t + \sigma B(t)),
$$
gdzie ruch Browna $B(t)$ stanowi źródło losowości. W projekcie ścieżki GBM powstają na podstawie wygenerowanego wektora wartości ruchu Browna o odpowiedniej kowariancji.  
Dla metod redukcji wariancji, takich jak stratyfikacja, stosujemy zmodyfikowane procedury generowania BM.

In [4]:
show_code("models/simulate_gbm_paths.py")

## 2.2. Definicja wypłaty opcji

W projekcie rozważamy dwie wypłaty: europejską oraz dyskretną barierową typu up-and-out.
Dla bariery $C < \infty$ wypłata wynosi zero, jeśli ścieżka ceny choć raz przekroczy poziom $C$; w przeciwnym razie jest to zdyskontowana wartość $(S(1) - K)_+$.  
Gdy $C = \infty$, otrzymujemy zwykłą opcję europejską.

In [6]:
show_code("payoffs/payoff_up_and_out_call.py")

## 2.3. Estymator Crude Monte Carlo

Podstawowy estymator Monte Carlo polega na generowaniu wielu niezależnych ścieżek GBM, obliczeniu wypłaty dla każdej z nich, a następnie uśrednieniu wyników. Jest to metoda referencyjna, do której porównujemy bardziej zaawansowane techniki redukcji wariancji.

In [7]:
show_code("methods/crude_monte_carlo.py")